This notebook shows how to use [dvc](https://dvc.org/) [experiments](https://github.com/iterative/dvc/wiki/Experiments) in model development. This example uses the [MNIST](http://yann.lecun.com/exdb/mnist/) data of handwritten digits and builds a classification model to predict the digit (0-9) in each image. The model is built in [pytorch](https://pytorch.org/) as a convolutional neural network with a simplified architecture, which should be able to quickly run on most computers.

### Get started

To get started, clone this repository and navigate to it.

The only other prerequisite is [docker](https://www.docker.com). Once docker is installed, build a docker image from the existing `Dockerfile` and run it:

```bash
docker build -t dvc-exp-mnist .
docker run -p 8888:8888 $(docker images -q dvc-exp-mnist)
```

To run the notebook, navigate to the link provided in the output that starts with `http://127.0.0.1:8888/`.

Next, set up git:

In [1]:
email = input("Your email:")
username = input("Your name:")

Your email:dave@iterative.ai
Your name:dave


In [2]:
%%bash -s {email} {username}
git config --global user.email "$1"
git config --global user.name "$2"

Finally, initialize dvc and enable the experiments feature:

In [3]:
%%bash
dvc init -f
dvc config --global core.experiments true

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


### Establish the pipeline DAG

Before experimenting, a dvc pipeline must be established (see the docs if you are new to dvc). Review the contents of `dvc.yaml` below to see the pipeline.

In [4]:
%%bash
cat dvc.yaml

stages:
  download:
    cmd: python download.py
    deps:
    - download.py
    outs:
    - data/MNIST
  train:
    cmd: python train.py
    deps:
    - data/MNIST
    - train.py
    params:
    - lr
    - weight_decay
    outs:
    - model.pt:
        checkpoint: true
    metrics:
    - metrics.yaml


The download stage gets the data using the `download.py` script. The train stage performs model training and evaluation on the downloaded data using the `train.py` script. The train stage uses the lr and weight_decay metrics defined in `params.yaml`. The model output is saved to `model.pt`, and the metrics are saved to `metrics.yaml`.

Execute the download stage to get the data.

In [5]:
%%bash
dvc repro download

Running stage 'download':
> python download.py
Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


 96%|█████████▋| 9543680/9912422 [00:01<00:00, 5273631.90it/s]
0it [00:00, ?it/s]
 57%|█████▋    | 16384/28881 [00:00<00:00, 162573.08it/s]

0it [00:00, ?it/s]

  1%|          | 16384/1648877 [00:00<00:12, 125725.60it/s]

 30%|██▉       | 491520/1648877 [00:00<00:00, 2541232.74it/s]

 55%|█████▍    | 901120/1648877 [00:00<00:00, 3190095.53it/s]

 92%|█████████▏| 1515520/1648877 [00:00<00:00, 4225321.45it/s]


0it [00:00, ?it/s]/opt/conda/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729002806/work/torch/csrc/utils/tensor_numpy.cpp:141.)


**IMPORTANT:** Be sure to run the `git add` command above and also `git commit` before running experiments. Anytime you modify the pipeline, be sure to `dvc repro` and track changes with git before running experiments.

In [6]:
%%bash
git add dvc.lock data/.gitignore
git commit -m "download data"

[2.0 7620c85] download data
 1 file changed, 1 insertion(+), 1 deletion(-)


### Run an experiment

Run an experiment with the default parameters defined in `params.yaml`.

In [7]:
%%bash
dvc exp run

Stage 'download' didn't change, skipping
Running stage 'train':
> python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '55c91e4'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '9ae3e29'.

To track the changes with git, run:

	git add dvc.lock train.py dvc.yaml data/MNIST .gitignore params.yaml

Reproduced experiment(s): exp-55405
Experiment results have been applied to your workspace.

To promote an experiment to a Git branch run:

	dvc exp branch <exp>



Review the output of the run, including identifying hashes, metrics, and parameters:

In [8]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.3751 │ 2.2356 │ 0.001 │ 0            │
│ 2.0           │ 12:54 AM │      - │  0.001 │ 0     │              │
│ │ ╓ exp-55405 │ 12:55 AM │ 0.3751 │ 2.2356 │ 0.001 │ 0            │
│ ├─╨ 55c91e4   │ 12:54 AM │ 0.2059 │ 2.2832 │ 0.001 │ 0            │
└───────────────┴──────────┴────────┴────────┴───────┴──────────────┘


Note that two experiments were run. These are checkpoints for the run. It's not necessary to have checkpoints for experiments, but they can be helpful for models that may be run for a number of epochs. See below for more information about how checkpoints work.

### Experiment with different parameters

Experiments can be run and compared with different parameters.

In [9]:
%%bash
dvc exp run --params weight_decay=0.1

Stage 'download' didn't change, skipping
Running stage 'train':
> python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'f6926f3'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'f773b3a'.

To track the changes with git, run:

	git add dvc.lock data/MNIST params.yaml dvc.yaml train.py

Reproduced experiment(s): exp-d4665
Experiment results have been applied to your workspace.

To promote an experiment to a Git branch run:

	dvc exp branch <exp>



In [10]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ 2.0           │ 12:54 AM │      - │  0.001 │ 0     │              │
│ │ ╓ exp-d4665 │ 12:55 AM │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ │ ╟ f773b3a   │ 12:55 AM │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ ├─╨ f6926f3   │ 12:55 AM │ 0.0966 │  2.301 │ 0.001 │ 0.1          │
│ │ ╓ exp-55405 │ 12:55 AM │ 0.3751 │ 2.2356 │ 0.001 │ 0            │
│ ├─╨ 55c91e4   │ 12:54 AM │ 0.2059 │ 2.2832 │ 0.001 │ 0            │
└───────────────┴──────────┴────────┴────────┴───────┴──────────────┘


Increasing `weight_decay` helped a bit. Next, try different `lr` parameters.

Experiments can be added in bulk to the queue and executed on demand (see the `-j` flag for parallel execution!).

In [11]:
%%bash
dvc exp run --params lr=0.01 --queue
dvc exp run --params lr=0.1 --queue

Queued experiment '2ba671b' for future execution.
Queued experiment 'fe29dbb' for future execution.


In [12]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ 2.0           │ 12:54 AM │      - │  0.001 │ 0     │              │
│ │ ╓ exp-d4665 │ 12:55 AM │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ │ ╟ f773b3a   │ 12:55 AM │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ ├─╨ f6926f3   │ 12:55 AM │ 0.0966 │  2.301 │ 0.001 │ 0.1          │
│ │ ╓ exp-55405 │ 12:55 AM │ 0.3751 │ 2.2356 │ 0.001 │ 0            │
│ ├─╨ 55c91e4   │ 12:54 AM │ 0.2059 │ 2.2832 │ 0.001 │ 0            │
│ ├── *fe29dbb  │ 12:56 AM │      - │    0.1 │ 0.1   │              │
│ └── *2ba671b  │ 12:56 AM │      - │   0.01 │ 0.1   │              │
└───────────────┴──────────┴────────┴────────┴───────┴──────────────┘


In [13]:
%%bash
dvc exp run --run-all

Stage 'download' didn't change, skipping
Stage 'download' didn't change, skipping
Running stage 'train':
> python train.py
Running stage 'train':
> python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '9170782'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '7c6c8a6'.

Reproduced experiment(s): exp-fe369
To apply the results of an experiment to your workspace run:

	dvc exp apply <exp>

To promote an experiment to a Git branch run:

	dvc exp branch <exp>



ERROR: failed to reproduce 'dvc.yaml': failed to run: python train.py, exited with -9
ERROR: Failed to reproduce experiment '2ba671b'


In [14]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ 2.0           │ 12:54 AM │      - │  0.001 │ 0     │              │
│ │ ╓ exp-fe369 │ 12:57 AM │ 0.0974 │ 2.3039 │ 0.1   │ 0.1          │
│ ├─╨ 9170782   │ 12:56 AM │ 0.1135 │ 2.3017 │ 0.1   │ 0.1          │
│ │ ╓ exp-d4665 │ 12:55 AM │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ │ ╟ f773b3a   │ 12:55 AM │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ ├─╨ f6926f3   │ 12:55 AM │ 0.0966 │  2.301 │ 0.001 │ 0.1          │
│ │ ╓ exp-55405 │ 12:55 AM │ 0.3751 │ 2.2356 │ 0.001 │ 0            │
│ ├─╨ 55c91e4   │ 12:54 AM │ 0.2059 │ 2.2832 │ 0.001 │ 0            │
│ └── *2ba671b  │ 12:56 AM │      - │   0.01 │ 0.1   │              │
└───────────────┴──────────┴────────┴────────┴───────┴──────────────┘


**FIXME:** Failed to run second experiment.

Let's try again to run the remaining queued experiments and to test that including the parallel execution flag doesn't break anything. Nothing should actually run in parallel here, since checkpoints need to be executed sequentially.

In [15]:
%%bash
dvc exp run --run-all -j 2

Stage 'download' didn't change, skipping
Running stage 'train':
> python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '92a0c3e'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '5837939'.

Reproduced experiment(s): exp-956e3
To apply the results of an experiment to your workspace run:

	dvc exp apply <exp>

To promote an experiment to a Git branch run:

	dvc exp branch <exp>



In [16]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ 2.0           │ 12:54 AM │      - │  0.001 │ 0     │              │
│ │ ╓ exp-956e3 │ 12:58 AM │ 0.1135 │ 2.3012 │ 0.01  │ 0.1          │
│ ├─╨ 92a0c3e   │ 12:58 AM │  0.207 │ 2.2979 │ 0.01  │ 0.1          │
│ │ ╓ exp-fe369 │ 12:57 AM │ 0.0974 │ 2.3039 │ 0.1   │ 0.1          │
│ ├─╨ 9170782   │ 12:56 AM │ 0.1135 │ 2.3017 │ 0.1   │ 0.1          │
│ │ ╓ exp-d4665 │ 12:55 AM │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ │ ╟ f773b3a   │ 12:55 AM │ 0.1076 │ 2.2926 │ 0.001 │ 0.1          │
│ ├─╨ f6926f3   │ 12:55 AM │ 0.0966 │  2.301 │ 0.001 │ 0.1          │
│ │ ╓ exp-55405 │ 12:55 AM │ 0.3751 │ 2.2356 │ 0.001 │ 0            │
│ ├─╨ 55c91e4   │ 12:54 AM │ 0.2059 │ 2.2832 │ 0.001 │ 0            │
└───────────────┴───

### Iteratively train using checkpoints

Use checkpoints to periodically save the model during training (as shown above), and to resume training from previously saved state. Resume training the experiment with the best accuracy.

In [17]:
%%bash
dvc exp show --sort-by acc --include-metrics acc

ERROR: failed to show experiments - Unknown sort column 'acc'


CalledProcessError: Command 'b'dvc exp show --sort-by acc --include-metrics acc\n'' returned non-zero exit status 1.

**FIXME**: Sort-by `acc` column fails.

In [22]:
%%bash
dvc exp show --include-metrics acc

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1076 │ 0.001 │ 0.1          │
│ 2.0           │ 12:54 AM │      - │ 0.001 │ 0            │
│ │ ╓ exp-956e3 │ 12:58 AM │ 0.1135 │ 0.01  │ 0.1          │
│ ├─╨ 92a0c3e   │ 12:58 AM │  0.207 │ 0.01  │ 0.1          │
│ │ ╓ exp-fe369 │ 12:57 AM │ 0.0974 │ 0.1   │ 0.1          │
│ ├─╨ 9170782   │ 12:56 AM │ 0.1135 │ 0.1   │ 0.1          │
│ │ ╓ exp-d4665 │ 12:55 AM │ 0.1076 │ 0.001 │ 0.1          │
│ │ ╟ f773b3a   │ 12:55 AM │ 0.1076 │ 0.001 │ 0.1          │
│ ├─╨ f6926f3   │ 12:55 AM │ 0.0966 │ 0.001 │ 0.1          │
│ │ ╓ exp-55405 │ 12:55 AM │ 0.3751 │ 0.001 │ 0            │
│ ├─╨ 55c91e4   │ 12:54 AM │ 0.2059 │ 0.001 │ 0            │
└───────────────┴──────────┴────────┴───────┴──────────────┘


Apply the top experiment.

In [23]:
%%bash
dvc exp apply exp-fe369

ERROR: Could not apply stash commit: Stash apply resulted in merge conflicts: Cmd('git') failed due to: exit code(1)
  cmdline: git stash apply 1e0e0f0691271937f135b79c1d9dde57ab8d473c
  stderr: '.ipynb_checkpoints/dvc-exp-mnist-checkpoint.ipynb already exists, no checkout
error: could not restore untracked files from stash'


CalledProcessError: Command 'b'dvc exp apply exp-fe369\n'' returned non-zero exit status 255.

Sometimes, we need to 

In [27]:
%%bash
dvc exp apply -f exp-fe369

Changes for experiment 'exp-fe369' have been applied to your current workspace.


In [25]:
%%bash
dvc exp apply 7c6c8a6

ERROR: Could not apply stash commit: Stash apply resulted in merge conflicts: Cmd('git') failed due to: exit code(1)
  cmdline: git stash apply 7e9736b7c48bab8c133c1ba2df0cc762aa8eba10
  stderr: '.ipynb_checkpoints/dvc-exp-mnist-checkpoint.ipynb already exists, no checkout
error: could not restore untracked files from stash'


CalledProcessError: Command 'b'dvc exp apply 7c6c8a6\n'' returned non-zero exit status 255.

Next, resume iterations from this experiment.

In [24]:
%%bash
dvc exp res -r 715fc6e

Stage 'download' didn't change, skipping
Running stage 'train':
> python train.py
Updating lock file 'dvc.lock'


ERROR: Error generating checkpoint, stage: 'train' will be aborted - Failed to set 'refs/exps/b9/f7373bf810065c85ac12f4af1f04a7dbae0b79/exp-98b33'


**FIXME**: Resuming experiment failed.

In [25]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1043 │ 2.3024 │ 0.001 │ 0.1          │
│ main          │ 11:19 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-e08c4 │ 11:27 AM │  0.101 │  2.303 │ 0.1   │ 0.1          │
│ ├─╨ 7251640   │ 11:26 AM │ 0.1135 │ 2.3026 │ 0.1   │ 0.1          │
│ │ ╓ exp-15d4f │ 11:23 AM │ 0.2012 │  2.071 │ 0.01  │ 0.1          │
│ ├─╨ 5fd4ddb   │ 11:22 AM │ 0.1914 │ 2.2331 │ 0.01  │ 0.1          │
│ │ ╓ exp-98b33 │ 11:21 AM │ 0.1009 │ 2.3028 │ 0.001 │ 0.1          │
│ │ ╟ 715fc6e   │ 11:21 AM │ 0.1009 │ 2.3028 │ 0.001 │ 0.1          │
│ ├─╨ 2b34f9f   │ 11:21 AM │ 0.1009 │ 2.3034 │ 0.001 │ 0.1          │
│ │ ╓ exp-ff372 │ 11:19 AM │ 0.1662 │ 2.2971 │ 0.001 │ 0            │
│ │ ╟ 8d76dda   │ 11:19 AM │ 0.1662 │ 2.2971 │ 0.001 │ 0            │
│ ├─╨ a9901cf   │ 11

In [26]:
%%bash
cat params.yaml

lr: 0.001
weight_decay: 0.1


The parameters from `dvc exp apply 31e8e72` were checked out into the workspace.

Compare this experiment to the initial one run with the default parameters:

In [29]:
%%bash
dvc exp diff 715fc6e a9901cf

Path          Metric    Value    Change
metrics.yaml  acc       0.1118   0.0109
metrics.yaml  loss      2.3028   -8.8215e-06

Path         Param         Value    Change
params.yaml  weight_decay  0        -0.1



### Persist models

Additonal epochs didn't improve accuracy, so commit the model iteration with peak accuracy. Checkout the experiment rev in dvc and then commit to git.

In [30]:
%%bash
dvc exp branch 715fc6e 2.0

Git branch '2.0' has been created from experiment 'exp-98b33'.
To switch to the new branch run:

	git checkout 2.0


In [31]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1043 │ 2.3024 │ 0.001 │ 0.1          │
│ main          │ 11:19 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-e08c4 │ 11:27 AM │  0.101 │  2.303 │ 0.1   │ 0.1          │
│ ├─╨ 7251640   │ 11:26 AM │ 0.1135 │ 2.3026 │ 0.1   │ 0.1          │
│ │ ╓ exp-15d4f │ 11:23 AM │ 0.2012 │  2.071 │ 0.01  │ 0.1          │
│ ├─╨ 5fd4ddb   │ 11:22 AM │ 0.1914 │ 2.2331 │ 0.01  │ 0.1          │
│ │ ╓ 2.0       │ 11:21 AM │ 0.1009 │ 2.3028 │ 0.001 │ 0.1          │
│ │ ╟ 715fc6e   │ 11:21 AM │ 0.1009 │ 2.3028 │ 0.001 │ 0.1          │
│ ├─╨ 2b34f9f   │ 11:21 AM │ 0.1009 │ 2.3034 │ 0.001 │ 0.1          │
│ │ ╓ exp-ff372 │ 11:19 AM │ 0.1662 │ 2.2971 │ 0.001 │ 0            │
│ │ ╟ 8d76dda   │ 11:19 AM │ 0.1662 │ 2.2971 │ 0.001 │ 0            │
│ ├─╨ a9901cf   │ 11

### Clean up experiments

In [32]:
%%bash
dvc exp gc --workspace

CalledProcessError: Command 'b'dvc exp gc --workspace\n'' returned non-zero exit status 1.

**FIXME**: `dvc exp gc --workspace` did not work.

In [33]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1043 │ 2.3024 │ 0.001 │ 0.1          │
│ main          │ 11:19 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-e08c4 │ 11:27 AM │  0.101 │  2.303 │ 0.1   │ 0.1          │
│ ├─╨ 7251640   │ 11:26 AM │ 0.1135 │ 2.3026 │ 0.1   │ 0.1          │
│ │ ╓ exp-15d4f │ 11:23 AM │ 0.2012 │  2.071 │ 0.01  │ 0.1          │
│ ├─╨ 5fd4ddb   │ 11:22 AM │ 0.1914 │ 2.2331 │ 0.01  │ 0.1          │
│ │ ╓ 2.0       │ 11:21 AM │ 0.1009 │ 2.3028 │ 0.001 │ 0.1          │
│ │ ╟ 715fc6e   │ 11:21 AM │ 0.1009 │ 2.3028 │ 0.001 │ 0.1          │
│ ├─╨ 2b34f9f   │ 11:21 AM │ 0.1009 │ 2.3034 │ 0.001 │ 0.1          │
│ │ ╓ exp-ff372 │ 11:19 AM │ 0.1662 │ 2.2971 │ 0.001 │ 0            │
│ │ ╟ 8d76dda   │ 11:19 AM │ 0.1662 │ 2.2971 │ 0.001 │ 0            │
│ ├─╨ a9901cf   │ 11